# Assignment 2 - Vehicle Routing Problem
JM0100-M-6 Business Analytics  
Myrthe Wouters  
u1273195

In [47]:
# Global imports
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import math
import operator
from copy import deepcopy
import random
import copy
from deap import base, creator, tools, algorithms
from elitism import eaSimpleWithElitism

In [48]:
# Load data
STORES = pd.read_excel('Data Excercise 2 - EMTE stores - BA 2020-1.xlsx', index_col='City Nr.')

In [49]:
STORES.head()

,Name,Address,Postal code,City,Lat,Long,Type
City Nr.,,,,,,,
0,EMTE HEADQUARTERS VEGHEL,CORRIDOR 11,5466RB,VEGHEL,51.606702,5.528046,NaN
1,EMTE ARKEL,DR H DE VRIESPLN 14,4241BW,ARKEL,51.864000,4.993040,Coop
2,EMTE ARNEMUIDEN FR,CLASINASTR 5,4341ER,ARNEMUIDEN,51.500010,3.677280,Jumbo
3,EMTE BATHMEN FR,LARENSEWG 18,7437BM,BATHMEN,52.249060,6.289990,Jumbo
4,EMTE BEEK EN DONK,HEUVELPLN 73,5741JJ,BEEK EN DONK,51.529300,5.632300,Jumbo


## Exercise 2.1

Write a construction heuristic to create a starting solution for the VRP based on the Insertion Method. 

### Calculate rounded distances between every two locations
I will use a matrix to store the distances between every two stores. This makes my algorithms more efficient (especially for 2.2), because I do not have to call the haversine formula every time I want to compute the distance between two stores.

In [50]:
def calc_dist(loc1, loc2):
    '''Defines rounded distance (km) between two locations'''
    coords1 = STORES.loc[loc1, 'Lat'], STORES.loc[loc1, 'Long']
    coords2 = STORES.loc[loc2, 'Lat'], STORES.loc[loc2, 'Long']
    dist = round(haversine(coords1, coords2))
    return dist

In [51]:
# Save all distance in global variable DIST_MATRIX
DIST_MATRIX = np.zeros((len(STORES), len(STORES)))

for location_1 in STORES.index:
    for location_2 in STORES.index:
        DIST_MATRIX[location_1, location_2] = calc_dist(location_1, location_2)

### Classes
For this assignment, I use object-oriented programming. For exercise 2.1, I define three classes:
* Location: class for stores with their metadata as attributes.
* Route: class for routes with their individual attributes and methods.
* RoutePlanner: class for entire schedules (i.e., a combination of routes that together visit every store exactly once) with attributes and methods to plan a schedule.

#### Location Class

In [52]:
class Location:
    
    def __init__(self, nr, name, lat, long, store_type):
        self.nr = nr
        self.name = name
        self.lat = lat
        self.long = long
        self.store_type = store_type
        self.visited = False
        self.distance_hq = DIST_MATRIX[0, nr]
    
    @property
    def is_jumbo(self):
        '''Boolean value that shows if location is a Jumbo location'''
        return True if self.store_type=='Jumbo' else False

In [53]:
def get_params(df, index):
    '''
    Function to get all necessary variables from STORES dataframe to instantiate a Location instance for every store
    '''
    params = df.iloc[index][['Name', 'Lat', 'Long', 'Type']].values
    name, lat, long, store_type = params
    return index, name, lat, long, store_type

#### Define Route Class

As mentioned before, I define a Route class for routes with their individual attributes and methods. 

*There are a few important attributes, properties and methods of the Route class to highlight:*

* the `farthest_store` attribute of a route is the location that is farthest a way from the HQ in that route
* the `inner_route` attribute of a route is a list of locations without start and end at HQ. 
* the `full_route` property of a route is a list of locations representing the entire route, including start and end at HQ. 
    * For example, if a route visits stores with numbers 4, 8 and 10 in that order. The `inner_route` of this route is `[4, 8, 10]`, while the `full_route` is `[0, 4, 8, 10, 0]`.
* the `all_hours_constraints` property defines if the route meets the constraints of John's working hours and the opening hours of the stores

In [54]:
class Route:
   
    # Set class variables
    max_working_mins = 11*60
    max_opening_mins = 8*60
    speed_kmh = 90
    
    def __init__(self, farthest_store, hq):
        self.inner_route=[] # inner_route is defined as the route without start and end at HQ
        self.farthest_store = farthest_store
        self.hq = hq
    
    @property
    def meeting_time(self):
        '''Defines the meeting time at individiual stores in the route'''
        return 30 if self.farthest_store.is_jumbo else 20
    
    @property
    def full_route(self):
        '''Defines full route of a route, including the start and finish at headquarters'''
        full_route = self.inner_route.copy()
        full_route.insert(0, self.hq)
        full_route.append(self.hq)
        return full_route
    
    @property
    def total_distance(self):
        '''Defines total distance in km of the route'''
        return sum(self.distances(full_route=True))
    
    @property
    def cumsum(self):
        '''
        Defines cumulative sum of distances at each location. 
        We need this cumulative sum in order to convert the final schedule to an Excel file.
        '''
        cumsum = list(np.cumsum(self.distances(full_route=True)))
        cumsum.insert(0, 0)
        return cumsum
    
    @property
    def total_visit_time(self):
        '''Defines total time spend in all stores of route together'''
        return len(self.inner_route) * self.meeting_time
    
    @property
    def working_hours_constraint(self):
        '''
        Defines if route meets the constraint of John's working hours. John cannot work more than 11 hours = 11*60 minutes per day.
        Hence, this method checks if the total travel time of the route and the total visit time at the stores does not exceed 11 hours.
        '''
        return (self.travel_time(full_route=True) + self.total_visit_time) <= Route.max_working_mins
    
    @property
    def opening_hours_constraint(self):
        '''
        Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route.
        This means that traveling time from the first store to the last store in the route (i.e., the inner route) plus total visiting time
        at the stores does not exceed 8 hours.
        '''
        return (self.travel_time(full_route=False) + self.total_visit_time) <= Route.max_opening_mins
    
    @property
    def all_hours_constraints(self):
        '''Defines if route meets both constraints John's working hours and visiting hours at every store'''
        return self.working_hours_constraint and self.opening_hours_constraint
    
    @staticmethod
    def dist_to_min(km):
        '''Defines the time of an amount of km travelled'''
        speed_kmm = (Route.speed_kmh)/60
        minutes = round(km/speed_kmm)
        return minutes

    def distances(self, full_route=False):
        '''
        Defines all distances between consecutive locations in a route.
        The argument full_route can be set to either True or False. False is default. If full_route is set to False, this method defines the 
        distances between consecutive locations in the inner route (i.e., excluding start and end at HQ). While if full_route is set to True, 
        this method defines the distances between consecutive locations in the full route (i.e., including start and end at HQ).
        '''
        if full_route:
            distances = [DIST_MATRIX[self.full_route[idx].nr, self.full_route[idx+1].nr] for 
                         idx, _ in enumerate(self.full_route[:-1])]
        else:    
            distances = [DIST_MATRIX[self.inner_route[idx].nr, self.inner_route[idx+1].nr] for 
                         idx, _ in enumerate(self.inner_route[:-1])]
        return distances
    
    def travel_time(self, full_route=False):
        '''
        Defines the total travel time of the route, i.e., the time spend travelling in driving to stores in route.
        The argument full_route can be set to either True or False. False is default. If full_route is set to False, this method defines the 
        travel time of the inner route (i.e., excluding start and end at HQ). While if full_route is set to True, this method defines the 
        travel time of the full route (i.e., including start and end at HQ).
        '''
        minutes_between_locs = [Route.dist_to_min(dist) for dist in self.distances(full_route=full_route)]
        travel_time = sum(minutes_between_locs)
        return travel_time
    
    def insert(self, pos, item):
        '''Insert location at given position in a route'''
        self.inner_route.insert(pos, item)
    
    def remove(self, item):
        '''Remove location from a route'''
        self.inner_route.remove(item)

#### RoutePlanner class
This class includes attributes and methods to generate a schedule (i.e., starting solution for VRP) through using a construction heuristic based on the Insertion Method.

*The generation of a schedule through the RoutePlanner class is implemented in the following way:*

Start with $j=1$
1. Select the farthest unplanned store based on distance from headquarters and plan this store in a new route $r_j$ 
2. Sort all potential unplanned stores (i.e., of same type (Jumbo or Coop/other) as farthest store) in increasing distance from farthest store
3. Try to plan each potential unplanned store $s_i$ at every position in route $r_j$. If there are feasible positions for $s_i$, insert $s_i$ at the best feasible position (i.e., the one that leads to the lowest total distance of route $r_j$). If there is no feasible positions for $s_i$, skip $s_i$ for route $r_j$
4. As long as there are unplanned stores left to visit, $j=j+1$ and continue at step 1.

Step 1-3 for one iteration are implemented in the method `plan_route` and the iterative algorithm is implemented in the method `solve`.

*There are a few important attributes, properties and methods of the RoutePlanner class to highlight:*

* the `locations` attribute contains a list of all 133 locations to visit in the entire schedule
* the `hq` attribute represents the headquarters
* the `loc_distances_hq` property defines a sorted list of all locations in decreasing distance from headquarters
* the `current_farthest_store` property defines an unplanned store $s$ that is farthest away from the headquarters at the start of iteration j
* the `potential_stores` method defines potential stores to visit given a particular farthest store. Potential stores include unplanned stores with same store type as the farthest store
* the `dist_farthest_store` method defines a sorted list of potential stores based on distance from farthest store
* the `plan_route` method executes the construction heuristic for one iteration, i.e., defines one route
* the `solve` method executes the entire construction heuristic and creates the schedule 

In [55]:
class RoutePlanner:
    
    def __init__(self, locations):
        self.locations = locations[1:]
        self.hq = locations[0]
        
    @property
    def loc_distances_hq(self):
        '''Sorts stores in decreasing distance from headquarter'''
        sorted_dist = sorted(self.locations, key=lambda x: x.distance_hq, reverse=True)
        return sorted_dist
    
    @property
    def to_visit(self):
        '''Defines the list of stores that still need to be visited in a future route'''
        stores_to_visit = [loc for loc in self.loc_distances_hq if loc.visited==False]
        return stores_to_visit
    
    @property
    def current_farthest_store(self):
        '''Defines current farthest store from HQ that has not been visited yet'''
        farthest_store = [loc for loc in self.loc_distances_hq if loc.visited==False][0]
        return farthest_store
    
    @property
    def stores_left(self):
        '''Defines if there are any stores left to visit'''
        return len(self.to_visit)>0
    
    def potential_stores(self, farthest_store):
        '''
        Defines potential stores for a given route with given farthest location.
        Potential stores for a route can be seen as stores that have the same store type as the farthest store (Jumbo or Coop/other) and that
        have not been visited in an earlier route (on an earlier day).
        '''
        if farthest_store.is_jumbo:
            potential_stores = [loc for loc in self.to_visit if loc.store_type=='Jumbo' and loc!=farthest_store]
        else: 
            potential_stores = [loc for loc in self.to_visit if loc.store_type!= 'Jumbo' and loc!=farthest_store]
        return potential_stores

    def dist_farthest_store(self, farthest_store):
        '''Sorts all the remaining unplanned stores in increasing distance from the current farthest store'''
        distances = [(DIST_MATRIX[farthest_store.nr, potential_store.nr], potential_store) 
                     for potential_store in self.potential_stores(farthest_store)]
        distances.sort(key=operator.itemgetter(0))
        sorted_dist = [potential_store for dist, potential_store in distances]
        return sorted_dist
    
    def plan_route(self, route):
        '''Plans individual routes, starting at the current farthest store'''
        
        # Insert current farthest store to be visited in a route
        route.insert(0, route.farthest_store)
        
        # All potential locations in ascending order
        for i in self.dist_farthest_store(route.farthest_store):
            best_dist = math.inf
            best_pos = None
            
            # Try location on every position in route
            for pos in range(len(route.inner_route)+1):
                
                # Insert location at position
                route.insert(pos, i)
                
                # Check if location at given position meets constraints and;
                # Check if this position is better than all previous viable positions for this location
                if route.all_hours_constraints and route.total_distance<best_dist:
                    best_dist = route.total_distance
                    best_pos = pos
                
                # Remove location from route
                route.remove(i)
            
            # If there is a viable position, insert location at best viable position in route
            if best_pos!=None:
                route.insert(best_pos, i)
        
        # Set all locations that are inserted to route to visited
        for loc in route.inner_route:
            loc.visited = True
            
    def solve(self):
        '''Solves VRP for all locations, i.e., makes final schedule through construction heuristic based on Insertion Method'''
        
        # Set route number
        j = 0
        routes = {} # Empty dict to store schedule
        
        # While there are unplanned stores, we should create a new route
        while self.stores_left:
            j += 1 # Update route number
            farthest_store = self.current_farthest_store # Assign current farthest store
            route = Route(farthest_store, self.hq) # Instantiate route instance with current farthest store
            self.plan_route(route) # Plan this route
            routes[j] = route # Store planned route with route nr as key in dict
            
        return routes

### Define function to save a schedule to an Excel file

In [56]:
def to_Excel(planned_routes, exercise_nr, save=True, file_name=None):
    '''Creates dataframe with needed information from solved VRP, saves to Excel file if needed'''
    
    # Dictionary to store data from all routes
    data = {'Route Nr.': [], 
            'City Nr.': [],
            'City Name': [], 
            'Total Distance in Route (km)': [],
            'Total Distance (km)': []}
    
    # Define total kilometers traveled at start of the route in previous routes
    total_km_at_start = 0
    
    # All routes in planned routes
    for nr, route in planned_routes.items():
        cumsum_loc = 0  # Define current cumulative sum location
        
        # Add data for location to dictionary
        # Because routes are instances of Route class in Exercise 1, but of RouteDLL class in Exercise 2,
        # We need to make an if-statement here
        if exercise_nr == 1:
            for location in route.full_route:
            
                data['Route Nr.'].append(nr)
                data['City Nr.'].append(location.nr)
                data['City Name'].append(location.name)
                data['Total Distance in Route (km)'].append(route.cumsum[cumsum_loc])
                data['Total Distance (km)'].append(route.cumsum[cumsum_loc] + total_km_at_start)
                
                cumsum_loc += 1 # Update current cumulative sum location
        
        # For Exercise 2, routes are instances of RouteDLL class (doubly linked lists)
        if exercise_nr == 2:
            temp = route.start # Initialise temp
            
            while (temp):
                data['Route Nr.'].append(nr)
                data['City Nr.'].append(temp.data.nr)
                data['City Name'].append(temp.data.name)
                data['Total Distance in Route (km)'].append(route.cumsum[cumsum_loc])
                data['Total Distance (km)'].append(route.cumsum[cumsum_loc] + total_km_at_start)
                
                temp = temp.next
            
                cumsum_loc += 1 # Update current cumulative sum location
        
        # Update total kilometers traveled at start of the route in previous routes
        total_km_at_start += route.cumsum[-1] 
    
    df = pd.DataFrame.from_dict(data) # Save data to DataFrame
    
    # If needed, save data to Excel file
    if save:
        df.to_excel(file_name, index=False)
            
    return df

### Solution 2.1

In [57]:
# Instantiate location instances for each store
locations = [Location(*get_params(STORES, index)) for index, _ in STORES.iterrows()]

# Instantiate planner instance
planner = RoutePlanner(locations)
planned_routes_1 = planner.solve() # Create initial solution for VRP

# Save results to Excel file
df_q21 = to_Excel(planned_routes_1, 1, file_name='../excel-files/Ex2.1-1273195.xls')

print('The initial schedule solution has a distance of {} km and contains {} routes.'.format(
    sum([route.total_distance for route in planned_routes_1.values()]), len(planned_routes_1)))

The initial schedule solution has a distance of 2985.0 km and contains 11 routes.


## Exercise 2.2
Write a local search improvement heuristic to improve the solution you have created in Exercise 2.1.

In [58]:
# Instantiate Location instances for all stores
locations = [Location(*get_params(STORES, index)) for index, _ in STORES.iterrows()]

In [59]:
# Define global variable HQ
HQ = locations[0]

For exercise 2.2, we have to do a lot of swap and move operations. Therefore, for efficiency reasons, I defined another Route class - RouteDLL - that stores a route as a doubly linked list.

### Classes

#### Node Class
As mentioned before, I decided to implement routes in exercise 2.2 as doubly linked list. For a doubly linked list, we need a Node class that stores data of a node along with previous and next pointers. The data of each node is an instance of Location class.

In [60]:
class Node:
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None

#### RouteDLL Class

As mentioned before, I define a RouteDLL class for routes with their individual attributes and methods. 

*There are a few important attributes, properties and methods of the RouteDLL class to highlight:*

* the `self.start` and `self.end` attributes of a DLL route represent the start and end and the HQ respectively.
* the `inner_distance_time` method calculates the distance and travel time of the inner route, where inner route has the same definition as in Exercise 2.1. 
* the `total_distance_time` method calculates the distance and travel time of the full route, where full route has the same definition as in Exercise 2.1. 
* the `is_valid` property defines if the route is valid, i.e., meets all of the following three constraints:
    * `store_type_constraint`: all stores in the route should be of the same type
    * `working_hours_constraints`: the entire duration of the route should be less than 11 hours
    * `opening_hours_constraints`: all stores in the route should be visited between 09:00-17:00
* the `insert_after` method inserts a node after a given other node in a route
* the `remove` method removes a node from a route

In [61]:
class RouteDLL:
    
    # Every route starts and ends at HQ
    hq_start = Node(HQ)
    hq_end = Node(HQ)
    
    # Class variables
    max_working_mins = 11*60
    max_opening_mins = 8*60
    speed_kmh = 90
    
    def __init__(self, farthest_store):
        self.start = RouteDLL.hq_start
        self.end = RouteDLL.hq_end
        self.start.next = self.end
        self.end.prev = self.start
        self.farthest_store = farthest_store
        self.is_jumbo = self.farthest_store.is_jumbo
        
    @property
    def meeting_time(self):
        '''Defines the meeting time at individiual stores in the route'''
        return 30 if self.is_jumbo else 20
    
    @property
    def length(self):
        '''Defines number of stores (including HQ) in the route'''
 
        temp = self.start # Initialise temp 
        count = 0 # Initialise count 
  
        # Loop while end of linked list is not reached 
        while (temp): 
            count += 1
            temp = temp.next
            
        return count 
    
    @property
    def full_route(self):
        '''Returns the order of store numbers in the full route as a list'''
        
        temp = self.start
        route = []
        
        while (temp):
            route += [temp.data.nr]
            temp = temp.next
            
        return route
    
    @property
    def total_visit_time(self):
        '''Defines total time spend in all stores of route together'''
        return ((self.length)-2) * self.meeting_time # Subtract 2 from self.length, as we do not have a meeting time at the HQ
    
    @property
    def store_type_constraint(self):
        '''
        Defines if all stores in route are of same type.
        More precisely, defines if all stores in a route are either all of type Jumbo or all of type Coop/other. 
        Returns True if a route meets this store type constraint, else False.
        '''
        temp = self.start.next
        
        while temp != self.end:
            if temp.data.is_jumbo != self.is_jumbo:
                return False
            temp=temp.next

        return True
    
    @property
    def working_hours_constraint(self):
        '''
        Defines if route meets the constraint of John's working hours. John cannot work more than 11 hours = 11*60 minutes per day.
        Hence, this method checks if the total travel time of the route and the total visit time at the stores does not exceed 11 hours.
        '''
        return (self.total_distance_time()['total_travel_time'] + self.total_visit_time) <= RouteDLL.max_working_mins
    
    @property
    def opening_hours_constraint(self):
        '''
        Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route.
        This means that traveling time from the first store to the last store in the route (i.e., the inner route) plus total visiting time
        at the stores does not exceed 8 hours.
        '''
        return (self.inner_distance_time()['total_travel_time'] + self.total_visit_time) <= RouteDLL.max_opening_mins
    
    @property
    def is_valid(self):
        '''
        Defines if route meets all constraints, that is:
        * Constraint of John's working hours (11 hours max)
        * Constraint of visiting hours at every store (9:00-17:00, that is, 8 hours max)
        * Constraint of every store in route being either of type Jumbo or of type Coop/other
        '''
        return self.working_hours_constraint and self.opening_hours_constraint and self.store_type_constraint
    
    @property
    def total_distance(self):
        '''Defines total distance of total route'''
        return self.total_distance_time()['total_distance']
    
    @property
    def cumsum(self):
        '''
        Defines cumulative sum of distances at each location.
        We need this cumulative sum in order to convert the final schedule to an Excel file.
        '''
        temp = self.start # Initialise temp
        distance = 0
        cumsum = []
        
        while temp.next:
            # Calculate distance between current and next node
            dist = DIST_MATRIX[temp.data.nr, temp.next.data.nr]
            
            # Add dist to distance
            distance += dist
            #Append cumulative distance until this node to cumsum list
            cumsum.append(distance)
            
            # Move to next node
            temp = temp.next
        
        # Insert cumsum of 0 at start at HQ
        cumsum.insert(0, 0)
        return cumsum
    
    @staticmethod
    def dist_to_min(km):
        '''Defines the time of an amount of km travelled'''
        speed_kmm = (RouteDLL.speed_kmh)/60
        minutes = round(km/speed_kmm)
        return minutes
    
    def total_distance_time(self):
        '''Defines total distance and travel time of the full route (i.e., route including start and end at HQ)'''
        temp = self.start # Initialise temp
        distance = 0
        travel_time = 0
        
        while temp.next:
            # Calculate distance and time between current and next node
            dist = DIST_MATRIX[temp.data.nr, temp.next.data.nr]
            time = RouteDLL.dist_to_min(dist)
            
            # Add dist and time to distance and travel_time
            distance += dist
            travel_time += time
            
            # Move to next node
            temp = temp.next
            
        return {'total_distance': distance,
                'total_travel_time': travel_time}
    
    def inner_distance_time(self):
        '''Defines total distance and travel time of inner route (i.e., route excluding start and end at HQ)'''
        temp = self.start.next # Initialise temp
        distance = 0
        travel_time = 0
        
        while temp.next != self.end:
            # Calculate distance and time between current and next node
            dist = DIST_MATRIX[temp.data.nr, temp.next.data.nr]
            time = RouteDLL.dist_to_min(dist)
            
            # Add dist and time to distance and travel_time
            distance += dist
            travel_time += time
            
            # Move to next node
            temp = temp.next
        
        return {'total_distance': distance,
                'total_travel_time': travel_time}    
        
    def insert_after(self, prev_node, new_node):
        '''Insert node at given position, i.e., after anothet node, in route'''
        
        # Check if prev_node exists
        if prev_node is None:
            print("This node doesn't exist in DLL")
            return
        
        # Make next of new node as next of prev_node
        new_node.next = prev_node.next
        
        # Make the next node of prev_node as new_node
        prev_node.next = new_node
        
        # Make the prev_node as previous of new_node
        new_node.prev = prev_node
        
        # Change previous of new_node's next node
        if new_node.next is not None:
            new_node.next.prev = new_node
            
    def remove(self, dele):
        '''Remove node from route'''
        
        # Change previous pointer of dele's next node:
        dele.next.prev = dele.prev 
        
        # Change next pointer of dele's previous node:
        dele.prev.next = dele.next
        
        # Set dele's previous and next pointers to None
        dele.next=None
        dele.prev=None

#### Solver Class 

The Solver Class includes attributes and methods to implement the local search improvement heuristic.

*The local search improvement heuristic through the Solver class is implemented in the following way:*
1. Define routes $r$ that changed since last iteration (initialise $r$ with all routes)
2. Compute all possible swaps and moves for each node in each route $r$ and save all swaps and moves that lead to an improvement and are valid
3. As long as there are swaps/move that lead to an improvement, compute the swap or move that leads to the highest improvement. If there are no swaps/moves that lead to an improvement anymore, break the loop
4. Update routes $r$ to routes that changed in this iteration
5. Delete saved improvements from nodes in route $r$ 
6. Continue at step 1

*There are a few important attributes, properties and methods of the Solver class to highlight:*
* the `planned_stores` attribute represents current schedule, instantiated with result of exercise 2.1
* the `all_routes_valid` property checks if all routes in the schedule are valid
* the `total_distance` property defines the total distance of the schedule
* the `all_swaps` and `all_moves` methods compute all swaps and moves respectively. Take an argument `changed_routes`. Only for nodes in these routes, all swaps/moves are computed
* the `optimize` method computes the entire local search improvement heuristic as described above

In [62]:
class Solver:
    
    def __init__(self, planned_routes):
        self.planned_routes = planned_routes
        
    @property
    def all_routes_valid(self):
        '''Checks if all planned routes in the schedule are valid, i.e., meets all constraints'''
        for route in self.planned_routes:
            if not route.is_valid:
                return False
        return True
    
    @property
    def total_distance(self):
        '''Defines total distance of all planned routes together'''
        dist = 0
        for route in self.planned_routes:
            dist += route.total_distance
        return dist
        
    @staticmethod
    def swap_nodes(node_1, node_2):
        '''Swaps node 1 with node 2, i.e., swap data of node 1 with data of node 2'''
        # Nothing to do if x and y are same 
        if node_1 == node_2: 
            return 
    
        temp = deepcopy(node_1.data)
        
        node_1.data = node_2.data
        node_2.data = temp
        
    @staticmethod
    def move_node_after(node, node_to_move_after):
        '''Move position of node to position after node_to_move_after'''
        
        # If node is is equal to node_to_move_after, do nothing
        if node == node_to_move_after:
            return
        
        # Remove node at original position
        ## Change node's previous node next pointer to node's next node
        node.prev.next = node.next
        
        ## Change node's next node previous pointer to node's previous node
        node.next.prev = node.prev
        
        # Insert node at new position
        ## Make next of new node as next of prev_node
        node.next = node_to_move_after.next
        
        ## Make the next node of prev_node as new_node
        node_to_move_after.next = node
        
        ## Make the prev_node as previous of new_node
        node.prev = node_to_move_after
        
        ## Change previous of new_node's next node
        node.next.prev = node
        
    @staticmethod
    def undo_move(route_org, route_curr, node, orig_before_node):
        '''Undo the move of node'''
        
        route_curr.remove(node)
        route_org.insert_after(orig_before_node, node)
    
    def all_swaps(self, changed_routes):
        '''
        Computes all possible swaps for all nodes in planned routes.
        This method only does forward swaps, as swapping node 1 with node 2 implies the same as swapping node 2 with node 1.
        Takes an argument changed_routes. The method only computes all possible swaps for nodes within changed_routes.
        '''
        
        # Dictionary to store the swaps that lead to new improvements in this iteration
        improvements = {}
        
        curr_dist = deepcopy(self.total_distance) # store the distance of all planned routes now together 
        
        # Check for each location
        # I use enumerate here, to check which route we are currently in, so that we can implement forward swaps only
        for idx_route, route in enumerate(self.planned_routes):
            # Set node to compute all swaps with
            curr_loc = route.start.next
            
            while curr_loc != route.end:
                # Compute only swaps with nodes in current route and routes after, in order to prevent doing double
                # the work
                for check_route in self.planned_routes[idx_route:]:
                    
                    # If none of the routes the two nodes are in changed compared to the last iteration,
                    # we have saved the improvement of the swap and thus do not have to compute it again
                    if (route not in changed_routes) and (check_route not in changed_routes):
                        continue
                    
                    # If any of the routes the two nodes are in changed, we have to recompute the improvement of the 
                    # swap
                    else: 
                        if route == check_route:
                            check_loc = curr_loc.next # Only check for nodes after current node
                        else:
                            check_loc = check_route.start.next
                        
                        while check_loc != check_route.end:
                            Solver.swap_nodes(curr_loc, check_loc) # swap two nodes
                        
                            improv = curr_dist - (self.total_distance) # check if there is an improvement
                            
                            # If the swap leads to an improvement and results in all valid routes 
                            # save improvement and details in improvements dictionary
                            if (improv > 0) and self.all_routes_valid:
                                improvements[(curr_loc, check_loc)] = {}
                                improvements[(curr_loc, check_loc)]['improv'] = improv
                                improvements[(curr_loc, check_loc)]['routes'] = (route, check_route)
                            
                            # Swap nodes back
                            Solver.swap_nodes(check_loc, curr_loc)
                            
                            # Move to next location to swap with
                            check_loc = check_loc.next
                            
                # Check all swaps for next location
                curr_loc = curr_loc.next
        return improvements
    
    def all_moves(self, changed_routes):
        '''
        Computes all possible swaps for all nodes in planned routes.
        Takes an argument changed_routes. The method only computes all possible moves for nodes within changed_routes.
        '''
        
        # Dictionary to store the moves that lead to new improvements in this iteration
        improvements = {}
        
        curr_dist = deepcopy(self.total_distance) # store the distance of all planned routes now together 
        
        # Check for each location
        for route in self.planned_routes:
            # Set node to compute all moves for
            curr_loc = route.start.next
            
            while curr_loc != route.end:
                # Save the node before the current node we are looking at, in order to be able to undo the move
                orig_before_node = curr_loc.prev
                
                for check_route in self.planned_routes:
                    # Set node we will move curr_loc after
                    move_after = check_route.start
                    
                    # If none of the routes the two nodes are in changed compared to the last iteration,
                    # we have saved the improvement of the move and thus do not have to compute it again
                    if (route not in changed_routes) and (check_route not in changed_routes):
                        continue
                    
                    else:
                        while move_after != check_route.end:
                        
                            Solver.move_node_after(curr_loc, move_after) # move curr_loc after move_after

                            improv = curr_dist - (self.total_distance) # check if there is an improvement
                            
                            # If the move leads to an improvement and results in all valid routes,
                            # save improvement and details in improvements dictionary
                            if (improv > 0) and self.all_routes_valid:
                                improvements[(curr_loc, move_after)] = {}
                                improvements[(curr_loc, move_after)]['improv'] = improv
                                improvements[(curr_loc, move_after)]['routes'] = (route, check_route)
                            
                            # Undo the move
                            solver.undo_move(route, check_route, curr_loc, orig_before_node)
                            
                            # Move to next location to move after
                            move_after = move_after.next
                
                # Check all moves for next location
                curr_loc = curr_loc.next

        return improvements
    
    def optimize(self):
        '''
        This method executes the local search improvement heuristic.
        In order to make the code more efficient, I implemented - as suggested - some clever logic such that we do not have to compute all 
        possible move and swap options again in each round, but update only those for which the corresponding routes are changed in the 
        previous round.
        This logic is implemented by storing the swap and move improvements respectively in the swap_improv and move_improv dictionaries. 
        As soon as I implemented the best move or swap, I store the changed routes through that move or swap in the variable changed_routes.
        Then, I delete swaps and moves with nodes that were in changed_routes from the the swap_improv and move_improv dictionary. In the next
        iteration, I update the move and swap improvements dictionary by computing all swaps and moves for the routes in changed_routes. For the
        swaps and moves within routes that have not changed, the swap_improv and move_improv still store all swaps and moves that lead to an 
        improvement and therefore do not require new computations.
        '''
        
        operations = 0 # Initialise nr of operation to lead to optimal result
        swap_improv = {} # Dictionary to save all swap improvements
        move_improv = {} # Dictionary to save all move improvements
        
        # Initialise routes that changed since last iteration. For the first iteration, we have not computed any moves or swaps yet.
        # Therefore, we have to compute all possible moves and swaps for each route.
        changed_routes = self.planned_routes 
        
        while True:
            # Compute all move and swaps for this iteration
            iter_swap_improv = self.all_swaps(changed_routes)
            iter_move_improv = self.all_moves(changed_routes)
            
            # Update dictionaries with swap and move improvements with improvements of changed routes 
            # during last iteration
            swap_improv.update(iter_swap_improv)
            move_improv.update(iter_move_improv)
            
            # Sort swap improvements on improvement value decreasing
            swap_improv_list = [(values['improv'], swap, values['routes']) for swap, values in swap_improv.items()]
            swap_improv_sorted = sorted(swap_improv_list, key=operator.itemgetter(0), reverse = True)
            
            # If there is a swap that leads to an improvement, set best_swap
            if len(swap_improv_sorted) > 0:
                # The best swap improvement is the first swap tuple in the swap_improv_sorted list. 
                best_swap = swap_improv_sorted[0] 
            else:
                best_swap = None # Else set to None, as there is no swap improvement
            
            # Sort move improvements on improvement value decreasing
            move_improv_list = [(values['improv'], move, values['routes']) for move, values in move_improv.items()]
            move_improv_sorted = sorted(move_improv_list, key=operator.itemgetter(0), reverse = True)
            
            # If there is a move that leads to an improvement, set best_move 
            if len(move_improv_sorted) > 0:
                # The best move improvement is the first move tuple in the move_improv_sorted list. 
                best_move = move_improv_sorted[0]
            else:
                best_move = None # Else set to None, as there is no move improvement
            
            # If there is no swap or move improvement anymore, we have found the optimal solution of the local search improvement heuristic
            # Hence, we can break the loop.
            if (len(swap_improv) + len(move_improv))==0:
                break
            
            # If the best improvement is a move improvement, compute the best move
            if best_move:
                if (not best_swap) or (best_move[0] > best_swap[0]):
                    # The second element in the best move tuple stores another tuple with the node to move and the node to move it after.
                    self.move_node_after(*best_move[1])
                    operations+=1 # Increase number of operations by 1
                    print('Moved {} after {}'.format(best_move[1][0].data.name, best_move[1][1].data.name))
                
                    # Keep track of the routes that have changed during the move operation
                    changed_routes = list(best_move[2])
                
            
            # If the best improvement is a swap improvement, compute the best swap
            if best_swap:
                if (not best_move) or (best_swap[0] >= best_move[0]):
                    # The second element in the best swap tuple stores another tuple with the two nodes to swap.
                    self.swap_nodes(*best_swap[1])
                    operations+=1 # increase number of operations by 1
                    print('Swapped {} with {}'.format(best_swap[1][0].data.name, best_swap[1][1].data.name))
                
                    # Keep track of the routes that have changed during the swap operation
                    changed_routes = list(best_swap[2])
                
            # Delete moves in changed_routes from move_improv dictionary, as they have to be computed again in 
            # next iteration
            for move in move_improv_sorted:
                # For each move tuple in the move_improv_sorted list, the routes that the two nodes belong to are stored in 
                # the second element of this tuple. Hence, for each move improvement, check if any of the routes the nodes were in 
                # have changed. If so, delete from move_improv dictionary as they need to be updated in next iteration.
                if any(True for route in changed_routes if route in move[2]):
                    del move_improv[move[1]]
                
            # Delete swaps in changed_routes from swap_improv dictionary, as they have to be computed again in 
            # next iteration
            for swap in swap_improv_sorted:
                # For each swap tuple in the swap_improv_sorted list, the routes that the two nodes belong to are stored in 
                # the second element of this tuple. Hence, for each swap improvement, check if any of the routes the nodes were in 
                # have changed. If so, delete from swap_improv dictionary as they need to be updated in next iteration.
                if any(True for route in changed_routes if route in swap[2]):
                    del swap_improv[swap[1]]
                        
        print('\nOptimized distance after local search improvement heuristic = {} in {} move/swap operations'.format(
            solver.total_distance, operations))
        
        # Make dictionary with route number and key and RouteDLL instance as value
        routes = {}
        i = 1
        for route in self.planned_routes:
            routes[i] = route
            i += 1
        
        return routes

### Set Route instances of Ex 2.1 (initial solution) to RouteDLL instances of Ex 2.2
In order to execute the local search improvement heuristic in exercise 2.2, I have to set the routes in the initial solution of exercise 2.1 to RouteDLL instances.

In [63]:
planned_routes_1_dll = []

for route in planned_routes_1.values():
    route_dll = RouteDLL(route.farthest_store)
    
    curr = route_dll.start # Initialise for insertion
    
    # Insert each location in inner route of route to route_dll
    for loc in route.inner_route:
        node_loc = Node(loc)
        route_dll.insert_after(curr, node_loc)
        curr = node_loc
    
    # Append a deep copy of the RouteDLL instance to a list of planned routes of type RouteDLL
    planned_routes_1_dll.append(deepcopy(route_dll))

### Solution 2.2

In [64]:
# Instantiate solver instance for local search improvement heuristic with solution of exercise 2.1
solver = Solver(planned_routes_1_dll)
planned_routes_2 = solver.optimize() # Solve local search improvement heuristic

# Save results to Excel file
df_q21 = to_Excel(planned_routes_2, 2, file_name='../excel-files/Ex2.2-1273195.xls')

Swapped EMTE HOEK FR with EMTE SEROOSKERKE FR
Moved EMTE OOLTGENSPLAAT FR after EMTE OUDENBOSCH
Swapped EMTE GROENLO with EMTE RIJSSEN VEENESLAGEN
Swapped EMTE RIJSSEN VEENESLAGEN with EMTE BATHMEN FR
Swapped EMTE UDENHOUT with EMTE KAATSHEUVEL
Moved EMTE LOON OP ZAND after EMTE HEADQUARTERS VEGHEL
Moved EMTE GROENLO after EMTE HEADQUARTERS VEGHEL
Swapped EMTE ENTER FR with EMTE RIJSSEN OPSTALLSTR
Moved EMTE Dongen after EMTE UDENHOUT
Moved EMTE TILBURG WAGNERPLEIN after EMTE UDENHOUT
Swapped EMTE OSSENDRECHT FR with EMTE PUTTE (NB)
Moved EMTE SCHAIJK after EMTE WIJCHEN
Swapped EMTE VLISSINGEN FR with EMTE KOUDEKERKE FR
Swapped EMTE ST MICHIELSGESTEL with EMTE DEN BOSCH
Moved EMTE RAVENSTEIN after EMTE HEADQUARTERS VEGHEL
Swapped EMTE LOBITH with EMTE CUIJK

Optimized distance after local search improvement heuristic = 2877.0 in 16 move/swap operations


## Exercise 2.3

In [65]:
STORES['is_jumbo'] = STORES.apply(lambda row: row['Type']=='Jumbo', axis=1) 
STORES.head()

,Name,Address,Postal code,City,Lat,Long,Type,is_jumbo
City Nr.,,,,,,,,
0,EMTE HEADQUARTERS VEGHEL,CORRIDOR 11,5466RB,VEGHEL,51.606702,5.528046,NaN,False
1,EMTE ARKEL,DR H DE VRIESPLN 14,4241BW,ARKEL,51.864000,4.993040,Coop,False
2,EMTE ARNEMUIDEN FR,CLASINASTR 5,4341ER,ARNEMUIDEN,51.500010,3.677280,Jumbo,True
3,EMTE BATHMEN FR,LARENSEWG 18,7437BM,BATHMEN,52.249060,6.289990,Jumbo,True
4,EMTE BEEK EN DONK,HEUVELPLN 73,5741JJ,BEEK EN DONK,51.529300,5.632300,Jumbo,True


In [66]:
def calc_dist(loc1, loc2):
    '''Defines rounded distance (km) between two locations'''
    coords1 = STORES.loc[loc1, 'Lat'], STORES.loc[loc1, 'Long']
    coords2 = STORES.loc[loc2, 'Lat'], STORES.loc[loc2, 'Long']
    dist = round(haversine(coords1, coords2))
    return dist

In [67]:
def dist_to_min(km):
    """Defines the duration of a route in minutes"""
    speed_kmm = 90/60
    minutes = round(km/speed_kmm)
    return minutes

In [68]:
# Save all distance in global variable DIST_MATRIX
DIST_MATRIX = np.zeros((len(STORES), len(STORES)))
TIME_MATRIX = np.zeros((len(STORES), len(STORES)))

for location_1 in STORES.index:
    for location_2 in STORES.index:
        dist = calc_dist(location_1, location_2)
        DIST_MATRIX[location_1, location_2] = dist
        TIME_MATRIX[location_1, location_2] = dist_to_min(dist)

In [69]:
toolbox = base.Toolbox()
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.10)

In [70]:
solution_ex_2 = []
route_types = {}

route_nr = 0
for route in planned_routes_2.values():
    curr_route = []
    if route.is_jumbo:
        route_types[route_nr] = 'Jumbo'
    else:
        route_types[route_nr] = 'Coop/Other'
    temp = route.start.next
    while temp != route.end:
        curr_route.append(temp.data.nr)
        temp = temp.next
    solution_ex_2.append(curr_route)
    route_nr+=1

old_sol = deepcopy(solution_ex_2)
    
solution_ex_2, route_types

([[7, 40, 41, 34, 96, 75, 127, 48, 65, 79, 27, 126, 78],
  [56, 45, 2, 67, 66, 124, 59, 82, 133, 102, 57, 54],
  [64, 35, 97, 81, 87, 80, 61, 98, 52, 101, 83, 60, 109, 39],
  [38, 71, 32, 28, 33, 31, 29, 30, 15, 50, 51, 49, 37],
  [23, 22, 90, 73, 44, 95, 3, 16, 74, 131, 100],
  [116, 113, 18, 19, 86, 89, 108, 11, 10, 94, 114, 36],
  [5, 12, 9, 1, 53, 88, 43, 6, 92, 58, 130, 24, 111, 85],
  [110, 13, 72, 68, 62, 119, 132, 129, 104, 55, 20, 125, 106],
  [91, 63, 14, 8, 105, 103, 26, 70, 69, 123, 17, 112, 77],
  [107, 25, 121, 93, 118, 117, 21, 122, 47, 46, 4],
  [120, 115, 84, 99, 128, 42, 76]],
 {0: 'Coop/Other',
  1: 'Jumbo',
  2: 'Coop/Other',
  3: 'Jumbo',
  4: 'Jumbo',
  5: 'Jumbo',
  6: 'Coop/Other',
  7: 'Jumbo',
  8: 'Coop/Other',
  9: 'Jumbo',
  10: 'Jumbo'})

In [71]:
# HELPER FUNCTIONS FOR EVALUATE FUNCTION

HQ = locations[0].nr
MAX_WORKING_MINS = 11*60
MAX_OPENING_MINS = 8*60
MAX_OPENING_MINS = 8*60

def flatten_schedule(schedule_nested):
    return [loc for route in schedule_nested for loc in route]

def nest_schedule(flat_schedule, lengths):
    nested_schedule = []
    start = 0
    for length in lengths:
        nested_schedule.append(flat_schedule[start:(start+length)])
        start+=length
    return nested_schedule
        

def total_distance(route, full_route=False):
    route_to_check = deepcopy(route)
    
    if full_route:
        route_to_check.insert(0, HQ)
        route_to_check.append(HQ)
    distances = [DIST_MATRIX[route_to_check[idx], route_to_check[idx+1]] 
                 for idx, _ in enumerate(route_to_check[:-1])]
    
    return sum(distances)
    

def travel_time(route, full_route=False):
    """
    Defines the total travel time of the route, i.e., the time spend travelling in driving to stores in route
    """
    route_to_check = deepcopy(route)
    
    if full_route:
        route_to_check.insert(0, HQ)
        route_to_check.append(HQ)

    travel_times = [TIME_MATRIX[route_to_check[idx], route_to_check[idx+1]]
                    for idx, _ in enumerate(route_to_check[:-1])]
    
    return sum(travel_times)

def visiting_time(route):
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    time = 30 if is_jumbo else 20
    return len(route)*time

def working_hours_constraint(route):
    """Defines if route meets the constraint of John's working hours"""
    return (travel_time(route, full_route=True) + visiting_time(route)) <= MAX_WORKING_MINS

def opening_hours_constraint(route):
    """Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route"""
    return (travel_time(route) + visiting_time(route)) <= MAX_OPENING_MINS

def valid_hours(route):
    return working_hours_constraint(route) and opening_hours_constraint(route)

def store_type_constraint(route):
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    
    for location in route[1:]:
        if STORES.loc[location, 'is_jumbo'] != is_jumbo:
            return False
    
    return True

def is_valid(route):
    return working_hours_constraint(route) and opening_hours_constraint(route) and store_type_constraint(route)

In [72]:
lengths = [len(route) for route in solution_ex_2]
flat = flatten_schedule(solution_ex_2)
nest_schedule(flat, lengths)

[[7, 40, 41, 34, 96, 75, 127, 48, 65, 79, 27, 126, 78],
 [56, 45, 2, 67, 66, 124, 59, 82, 133, 102, 57, 54],
 [64, 35, 97, 81, 87, 80, 61, 98, 52, 101, 83, 60, 109, 39],
 [38, 71, 32, 28, 33, 31, 29, 30, 15, 50, 51, 49, 37],
 [23, 22, 90, 73, 44, 95, 3, 16, 74, 131, 100],
 [116, 113, 18, 19, 86, 89, 108, 11, 10, 94, 114, 36],
 [5, 12, 9, 1, 53, 88, 43, 6, 92, 58, 130, 24, 111, 85],
 [110, 13, 72, 68, 62, 119, 132, 129, 104, 55, 20, 125, 106],
 [91, 63, 14, 8, 105, 103, 26, 70, 69, 123, 17, 112, 77],
 [107, 25, 121, 93, 118, 117, 21, 122, 47, 46, 4],
 [120, 115, 84, 99, 128, 42, 76]]

In [73]:
def swap_locations(schedule, idx_route1, idx_route2, idx_elem1, idx_elem2):
    schedule[idx_route1][idx_elem1], schedule[idx_route2][idx_elem2] = schedule[idx_route2][idx_elem2], schedule[idx_route1][idx_elem1]

In [74]:
def swap_with_random_OLD(schedule, idx_route, idx_elem, route_type):
    
    possible_routes_to_swap = [route for route, typ in route_types.items() if typ==route_type]
    idx_route_to_swap = random.choice(possible_routes_to_swap)
    if idx_route == idx_route_to_swap:
        possible_elems_to_swap = list(range(len(schedule[idx_route_to_swap])))
        possible_elems_to_swap.remove(idx_elem)
        idx_elem_to_swap = random.choice(possible_elems_to_swap)
    else:
        idx_elem_to_swap = random.choice(range(len(schedule[idx_route_to_swap])))
    
    swap_locations(schedule, idx_route, idx_route_to_swap, idx_elem, idx_elem_to_swap)
    
    all_valid = all(valid_hours(route) for route in schedule)
    if all_valid:
        dist = sum([otal_distance(route, full_route=True) for route in schedule])
    else: 
        dist=None
    swap_details = idx_route, idx_route_to_swap, idx_elem, idx_elem_to_swap
    
    swap_locations(schedule, idx_route_to_swap, idx_route, idx_elem_to_swap, idx_elem)
    
    return all_valid, dist, swap_details

In [75]:
def swap_with_random(schedule, idx_route, idx_elem, route_type):
    
    for i in range(50):
        possible_routes_to_swap = [route for route, typ in route_types.items() if typ==route_type]
        idx_route_to_swap = random.choice(possible_routes_to_swap)
        if idx_route == idx_route_to_swap:
            possible_elems_to_swap = list(range(len(schedule[idx_route_to_swap])))
            possible_elems_to_swap.remove(idx_elem)
            idx_elem_to_swap = random.choice(possible_elems_to_swap)
        else:
            idx_elem_to_swap = random.choice(range(len(schedule[idx_route_to_swap])))
    
        swap_locations(schedule, idx_route, idx_route_to_swap, idx_elem, idx_elem_to_swap)
    
        changed_routes_valid = all((valid_hours(schedule[idx_route]), valid_hours(schedule[idx_route_to_swap])))
        if not changed_routes_valid:
            swap_locations(schedule, idx_route_to_swap, idx_route, idx_elem_to_swap, idx_elem)
    
        else:
            break

In [76]:
def swap_with_random2(schedule, idx_route, idx_elem, route_type):
    
    possible_routes_to_swap = [route for route, typ in route_types.items() if typ==route_type]
    idx_route_to_swap = random.choice(possible_routes_to_swap)
    if idx_route == idx_route_to_swap:
        possible_elems_to_swap = list(range(len(schedule[idx_route_to_swap])))
        possible_elems_to_swap.remove(idx_elem)
        idx_elem_to_swap = random.choice(possible_elems_to_swap)
    else:
        idx_elem_to_swap = random.choice(range(len(schedule[idx_route_to_swap])))

    swap_locations(schedule, idx_route, idx_route_to_swap, idx_elem, idx_elem_to_swap)

    changed_routes_valid = all((valid_hours(schedule[idx_route]), valid_hours(schedule[idx_route_to_swap])))
    if changed_routes_valid:
        dist = sum([total_distance(route, full_route=True) for route in schedule])
    else: 
        dist=None
    swap_details = idx_route, idx_route_to_swap, idx_elem, idx_elem_to_swap
        
    swap_locations(schedule, idx_route_to_swap, idx_route, idx_elem_to_swap, idx_elem)
    
    return changed_routes_valid, dist, swap_details, 'swap'

In [77]:
#FOUT MET REMOVE EN DAN WEER INSERTEN
def move_locations(schedule, orig_idx_route, idx_route_to_move, elem, new_pos):
    schedule[orig_idx_route].remove(elem)
    schedule[idx_route_to_move].insert(new_pos, elem)

In [78]:
# FOUT MET REMOVE EN DAN WEER INSERTEN
def move_to_random(schedule, idx_route, idx_elem, elem, route_type):
    
    possible_routes_to_move = [route for route, typ in route_types.items() if typ==route_type]
    idx_route_to_move = random.choice(possible_routes_to_move)
    if idx_route == idx_route_to_move:
        possible_positions = list(range(len(schedule[idx_route_to_move])+1))
        possible_positions.remove(idx_elem)
        new_pos = random.choice(possible_positions)
    else:
        new_pos = random.choice(range(len(schedule[idx_route_to_move])+1))
    
    move_locations(schedule, idx_route, idx_route_to_move, elem, new_pos)
    
    changed_route_valid = valid_hours(schedule[idx_route_to_move])
    if changed_route_valid:
        dist = sum([total_distance(route, full_route=True) for route in schedule])
    else: 
        dist=None
    move_details = idx_route, idx_route_to_move, elem, new_pos
    
    move_locations(schedule, idx_route_to_move, idx_route, elem, idx_elem)
    
    return changed_route_valid, dist, move_details, 'move'

In [79]:
def mutate(individual, indpb):
    
    for idx_route, route in enumerate(individual):
        route_type = route_types[idx_route]
        for idx_elem, loc in enumerate(route):
            swap = random.randint(0,1)
            if random.random() < indpb:
                if swap:
                    swap_with_random(individual, idx_route, idx_elem, route_type)
                
                else:
                    move_informations = []
                    for i in range(100):
                        information = move_to_random(individual, idx_route, idx_elem, loc, route_type)
                        move_informations.append(information)
                    
                    valid_moves = [move_info[1:] for move_info in move_informations if move_info[0]]
                    if len(valid_moves):
                        best_move = sorted(valid_moves, key=lambda x: x[0])[0]
                        move_locations(individual, *best_move[1])
                        
    return individual,

In [80]:
def mutate2(individual, indpb):
    for idx_route, route in enumerate(individual):
        route_type = route_types[idx_route]
        for idx_elem, loc in enumerate(route):
            if random.random() < indpb:

                informations = []
                for i in range(100):
                    swap = random.randint(0,1)
                    if swap:
                        information = swap_with_random2(individual, idx_route, idx_elem, route_type)
                        informations.append(information)
                    else: 
                        information = move_to_random(individual, idx_route, idx_elem, loc, route_type)
                        informations.append(information)
                valid = [info[1:] for info in informations if info[0]]
                if len(valid):
                    best = sorted(valid, key=lambda x: x[0])[0]
                    if best[-1] == 'move':
                        move_locations(individual, *best[1])
                    else:
                        swap_locations(individual, *best[1])
                        
    return individual,

In [35]:
tryout = deepcopy(solution_ex_2)
mutate(tryout, 0.05)

([[7, 40, 41, 34, 96, 75, 127, 48, 65, 79, 27, 126, 78],
  [56, 45, 2, 67, 66, 124, 59, 82, 133, 102, 57, 54],
  [64, 97, 81, 60, 83, 101, 52, 98, 61, 80, 87, 35, 112, 77],
  [38, 71, 32, 28, 33, 31, 29, 30, 15, 50, 51, 49, 37],
  [23, 22, 90, 73, 95, 3, 16, 74, 131, 100],
  [116, 113, 18, 19, 86, 89, 108, 11, 10, 94, 114, 36],
  [5, 12, 9, 1, 53, 39, 109, 88, 43, 6, 92, 58, 130, 24, 111, 85],
  [110, 68, 72, 13, 62, 119, 132, 129, 104, 55, 20, 106],
  [103, 17, 123, 26, 70, 69, 105, 8, 14, 63, 91],
  [107, 25, 121, 93, 118, 117, 21, 122, 47, 46, 4],
  [120, 115, 44, 84, 99, 42, 125, 128, 76]],)

In [36]:
all([is_valid(route) for route in tryout])

True

In [81]:
def crossover(ind1, ind2):
    
    lengths_ind1 = [len(route) for route in ind1]
    lengths_ind2 = [len(route) for route in ind2]
    
    flat_ind1, flat_ind2 = flatten_schedule(ind1), flatten_schedule(ind2)
    flat_ind1_deap = [nr-1 for nr in flat_ind1]
    flat_ind2_deap = [nr-1 for nr in flat_ind2]
    
    for i in range(500):
        flat_child1_deap, flat_child2_deap = tools.cxPartialyMatched(flat_ind1_deap[:], flat_ind2_deap[:])
        flat_child1 = [nr+1 for nr in flat_child1_deap]
        flat_child2 = [nr+1 for nr in flat_child2_deap]
        
        child1 = nest_schedule(flat_child1, lengths_ind1)
        child2 = nest_schedule(flat_child2, lengths_ind2)
        
        if all(is_valid(route) for route in child1) and all(is_valid(route) for route in child2):
            return creator.Individual(child1), creator.Individual(child2)
    print('failed to mate')
    return ind1, ind2

In [82]:
sum([total_distance(route, full_route=True) for route in solution_ex_2])

2877.0

In [39]:
ind1 = deepcopy(solution_ex_2)
mutate(ind1, 0.05)
test1 = deepcopy(ind1)

In [40]:
ind2 = deepcopy(solution_ex_2)
mutate(ind2, 0.05)
test2 = deepcopy(ind2)

In [ ]:
child1, child2 = crossover(ind1, ind2)

In [43]:
ind1==child1, ind2==child2

(False, False)

In [55]:
same = 0
different = 0
for i in range(100):
    ind1 = deepcopy(solution_ex_2)
    mutate(ind1, 0.05)
    
    ind2 = deepcopy(solution_ex_2)
    mutate(ind2, 0.05)
    child1, child2 = crossover(ind1, ind2)
    if child1==ind1 and child2==ind2:
        same+=1 
    else: different+=1

same, different

(42, 58)

In [92]:
def evaluate_distance(individual):
    return sum([total_distance(route, full_route=True) for route in individual]), 

In [93]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

init_pop = [deepcopy(solution_ex_2) for i in range(200)]

def initIndividual(icls, content):
    return icls(content)

def initPopulation(pcls, ind_init, init_pop):
    for ind in init_pop:
        mutate(ind, 0.05)
    return pcls(ind_init(ind) for ind in init_pop)

toolbox = base.Toolbox()

toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, init_pop)

In [94]:
toolbox.register("evaluate", evaluate_distance)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate2, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=2)

In [95]:
def main():
    pop = toolbox.population_guess()
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    pop, log = eaSimpleWithElitism(pop, toolbox, cxpb=0.7, mutpb=0.3, ngen=10000, 
                                   stats=stats, halloffame=hof, verbose=True)
    
    return pop, log, hof

In [96]:
pop, log, hof = main()

gen	nevals	avg   	std    	min 	max 
0  	200   	3023.7	84.5302	2888	3331
1  	158   	3000.26	68.9083	2888	3275
2  	161   	2982.92	68.4414	2888	3247
3  	161   	2972.18	64.8815	2881	3211
4  	158   	2961.93	64.0968	2881	3253
5  	153   	2953.54	55.1026	2881	3137
6  	158   	2952.35	56.76  	2881	3186
7  	162   	2957.26	74.5679	2881	3310
8  	165   	2950.57	65.9538	2881	3224
9  	158   	2947.82	62.0751	2881	3158
10 	154   	2946.05	65.2944	2873	3238
11 	154   	2945.51	61.4445	2873	3155
12 	160   	2937.64	53.0963	2873	3146
13 	160   	2945.84	65.3327	2873	3229
14 	161   	2940.38	54.9979	2873	3182
15 	160   	2937.72	53.8299	2873	3193
16 	161   	2946.93	66.3362	2872	3215
17 	169   	2938.16	59.0745	2872	3229
18 	154   	2933.62	53.5802	2872	3172
19 	148   	2929.66	55.6781	2869	3185
20 	159   	2928.97	60.2545	2869	3248
21 	156   	2918.67	53.2413	2869	3163
22 	148   	2917.82	54.143 	2869	3099
23 	154   	2915.81	55.8673	2869	3198
24 	170   	2917.59	59.3869	2866	3138
25 	151   	2910.93	57.4545	2866	3195
26 

In [97]:
hof.items

[[[7, 40, 41, 34, 96, 75, 127, 48, 65, 79, 27, 126, 78],
  [56, 45, 2, 67, 66, 124, 59, 82, 133, 102, 57, 54],
  [12, 64, 97, 81, 87, 80, 61, 98, 52, 101, 83, 60, 109, 39],
  [38, 71, 32, 28, 33, 31, 29, 30, 15, 50, 51, 49, 37],
  [23, 22, 90, 73, 44, 95, 3, 16, 74, 131, 100],
  [76, 113, 18, 19, 86, 89, 108, 11, 10, 94, 114, 36],
  [5, 9, 1, 53, 88, 43, 6, 92, 58, 130, 24, 111, 85],
  [84, 110, 13, 72, 68, 62, 119, 132, 129, 104, 55, 116, 42],
  [91, 63, 14, 8, 105, 69, 70, 26, 123, 17, 35, 112, 77, 103],
  [107, 25, 121, 93, 118, 117, 21, 122, 47, 46, 4],
  [120, 115, 99, 125, 20, 128, 106]]]